### HR

In [ ]:
#A for  loop is used to loop through the different cross-validation sets
for i in range(1,6):
    #########################################
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    from tensorflow.python.keras.layers import Dense, Flatten
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.optimizers import Adam, SGD

#Create the data loaders
    
    train_ds = keras.utils.image_dataset_from_directory(
        directory=r'D:\Gaussian_blur_bicubic_interpolation\Classification\HR\{}\train'.format(i),
        labels='inferred',
        label_mode='binary',
        batch_size=32,
        color_mode="rgb",
        seed=123,
        image_size=(240, 240))


    test_ds = keras.utils.image_dataset_from_directory(
        directory=r'D:\Gaussian_blur_bicubic_interpolation\Classification\HR\{}\test'.format(i),
        labels='inferred',
        label_mode='binary',
        batch_size=32,
        color_mode="rgb",
        seed=123,
        image_size=(240, 240))



    ###############################################

    #Create the ResNet and train it with early stopping, the best performing modefl after 40 epochs is saved
    
    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import Flatten, Dense, Input
    from tensorflow.keras.optimizers import SGD
    import tensorflow as tf
    
    
    input_tensor = Input(shape=(240, 240, 3))
    pretrained_model = tf.keras.applications.ResNet50(
        include_top=False,
        pooling='avg',
        weights='imagenet',
        input_tensor=input_tensor
    )

    x = pretrained_model.output
    x = Flatten()(x)
    output_tensor = Dense(1, activation='sigmoid')(x)
    resnet_model = Model(inputs=input_tensor, outputs=output_tensor)

    for layer in pretrained_model.layers:
        layer.trainable = False

    resnet_model.compile(optimizer=SGD(learning_rate=0.01),loss='binary_crossentropy',metrics=['accuracy'])


    callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=40,  restore_best_weights=True)
    
    checkpoint = tf.keras.callbacks.ModelCheckpoint("HR_best_model.weights.hdf5", 
                    monitor="val_accuracy", mode="max", 
                    save_best_only=True, verbose=1)

    history = resnet_model.fit(train_ds, epochs=40, callbacks=[callback, checkpoint], validation_data = test_ds, verbose=0)
    
    resnet_model.load_weights("HR_best_model.weights.hdf5")

    ##################################################
    import numpy as np

    #Calculate the predications on the test
    
    y_pred = []
    y_true = []

    for img, labels in test_ds:
        group_labels = resnet_model.predict(img,verbose=0)
        for label in group_labels.tolist():
            y_pred.append(np.round(label[0]))
        labels = labels.numpy()
        for label in labels.tolist():
            y_true.append(label[0])

    ###################################################
    import csv
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    # Save values to CSV file
    header = ['Accuracy', 'Precision', 'Recall', 'F1-score']
    data = [accuracy, precision, recall, f1]

    csv_filename = 'performance_metrics\HR\cross_validation_{}_results.csv'.format(i)

    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerow(data)

    print(f'Metrics saved to {csv_filename}')

    ##################################################
    #Saving epoch and accuracy results into  a csv file
    counter = 1
    Accuracy = []
    Epoch = []
    Val_accuracy = []

    for value in history.history['accuracy']:
        Accuracy.append(round(value,2))
        Epoch.append(counter)
        counter += 1

    for value in history.history['val_accuracy']:
        Val_accuracy.append((round(value,2)))

    import csv

    csv_file = 'Epoch_SR_results\HR\cross_validation_{}_epcoch_SR_results.csv'.format(i)

    # Combine the two lists into a list of tuples
    data = zip(Epoch, Accuracy, Val_accuracy)

    # Write to the CSV file
    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)

        # Write the header
        writer.writerow(['Epoch', 'Accuracy', 'Val_accuracy'])

        # Write the data
        writer.writerows(data)

    print(f'CSV file "{csv_file}" created successfully.')


########################################################

import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt 

#Calculating the confusion matrix for the last cross-validation set.
matrix = confusion_matrix(y_pred,y_true)
 
#Computing the confusion matrix.
sns.heatmap(matrix, 
            annot=True,
            fmt='g', 
            xticklabels=['HGG', 'LGG'],
            yticklabels=['HGG','LGG'])
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.title('Confusion Matrix',fontsize=17)
plt.savefig('HR_plot.png')
plt.show()

###  SR

In [ ]:
for i in range(5,6):
    #########################################
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    from tensorflow.python.keras.layers import Dense, Flatten
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.optimizers import Adam, SGD

    train_ds = keras.utils.image_dataset_from_directory(
        directory=r'D:\Gaussian_blur_bicubic_interpolation\Classification\SR\{}\train'.format(i),
        labels='inferred',
        label_mode='binary',
        batch_size=32,
        color_mode="rgb",
        seed=123,
        image_size=(240, 240))


    test_ds = keras.utils.image_dataset_from_directory(
        directory=r'D:\Gaussian_blur_bicubic_interpolation\Classification\SR\{}\test'.format(i),
        labels='inferred',
        label_mode='binary',
        batch_size=32,
        color_mode="rgb",
        seed=123,
        image_size=(240, 240))



    ###############################################

    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import Flatten, Dense, Input
    from tensorflow.keras.optimizers import SGD
    import tensorflow as tf
    
    
    input_tensor = Input(shape=(240, 240, 3))
    pretrained_model = tf.keras.applications.ResNet50(
        include_top=False,
        pooling='avg',
        weights='imagenet',
        input_tensor=input_tensor
    )

    x = pretrained_model.output
    x = Flatten()(x)
    output_tensor = Dense(1, activation='sigmoid')(x)
    resnet_model = Model(inputs=input_tensor, outputs=output_tensor)

    for layer in pretrained_model.layers:
        layer.trainable = False

    resnet_model.compile(optimizer=SGD(learning_rate=0.01),loss='binary_crossentropy',metrics=['accuracy'])


    callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=40,  restore_best_weights=True)
    
    checkpoint = tf.keras.callbacks.ModelCheckpoint("SR_best_model.weights.hdf5", 
                    monitor="val_accuracy", mode="max", 
                    save_best_only=True, verbose=1)

    history = resnet_model.fit(train_ds, epochs=40, callbacks=[callback, checkpoint], validation_data = test_ds, verbose=0)
    
    resnet_model.load_weights("SR_best_model.weights.hdf5")

    ##################################################
    import numpy as np

    y_pred = []
    y_true = []

    for img, labels in test_ds:
        group_labels = resnet_model.predict(img,verbose=0)
        for label in group_labels.tolist():
            y_pred.append(np.round(label[0]))
        labels = labels.numpy()
        for label in labels.tolist():
            y_true.append(label[0])

    ###################################################
    import csv
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    # Save values to CSV file
    header = ['Accuracy', 'Precision', 'Recall', 'F1-score']
    data = [accuracy, precision, recall, f1]

    csv_filename = 'performance_metrics\SR\cross_validation_{}_results.csv'.format(i)

    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerow(data)

    print(f'Metrics saved to {csv_filename}')

    ##################################################
    #Saving epoch and accuracy results into  a csv file
    counter = 1
    Accuracy = []
    Epoch = []
    Val_accuracy = []

    for value in history.history['accuracy']:
        Accuracy.append(round(value,2))
        Epoch.append(counter)
        counter += 1

    for value in history.history['val_accuracy']:
        Val_accuracy.append((round(value,2)))

    import csv

    csv_file = 'Epoch_SR_results\SR\cross_validation_{}_epcoch_SR_results.csv'.format(i)

    # Combine the two lists into a list of tuples
    data = zip(Epoch, Accuracy, Val_accuracy)

    # Write to the CSV file
    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)

        # Write the header
        writer.writerow(['Epoch', 'Accuracy', 'Val_accuracy'])

        # Write the data
        writer.writerows(data)

    print(f'CSV file "{csv_file}" created successfully.')


########################################################

import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt 

#Calculating the confusion matrix.
matrix = confusion_matrix(y_pred,y_true)
 
#Computing the confusion matrix.
sns.heatmap(matrix, 
            annot=True,
            fmt='g', 
            xticklabels=['HGG', 'LGG'],
            yticklabels=['HGG','LGG'])
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.title('Confusion Matrix',fontsize=17)
plt.savefig('SR_plot.png')
plt.show()

### LR

In [ ]:
for i in range(1,6):
    #########################################
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    from tensorflow.python.keras.layers import Dense, Flatten
    from tensorflow.keras.models import Sequential, Model
    from tensorflow.keras.optimizers import Adam, SGD

    train_ds = keras.utils.image_dataset_from_directory(
        directory=r'D:\Gaussian_blur_bicubic_interpolation\Classification\LR\{}\train'.format(i),
        labels='inferred',
        label_mode='binary',
        batch_size=32,
        color_mode="rgb",
        seed=123,
        image_size=(60, 60))


    test_ds = keras.utils.image_dataset_from_directory(
        directory=r'D:\Gaussian_blur_bicubic_interpolation\Classification\LR\{}\test'.format(i),
        labels='inferred',
        label_mode='binary',
        batch_size=32,
        color_mode="rgb",
        seed=123,
        image_size=(60, 60))



    ###############################################
    
    
    
    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import Flatten, Dense, Input
    from tensorflow.keras.optimizers import SGD
    import tensorflow as tf
    
    
    input_tensor = Input(shape=(60, 60, 3))
    pretrained_model = tf.keras.applications.ResNet50(
        include_top=False,
        pooling='avg',
        weights='imagenet',
        input_tensor=input_tensor
    )

    x = pretrained_model.output
    x = Flatten()(x)
    output_tensor = Dense(1, activation='sigmoid')(x)
    resnet_model = Model(inputs=input_tensor, outputs=output_tensor)

    for layer in pretrained_model.layers:
        layer.trainable = False

    resnet_model.compile(optimizer=SGD(learning_rate=0.01),loss='binary_crossentropy',metrics=['accuracy'])


    callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=40,  restore_best_weights=True)
    
    checkpoint = tf.keras.callbacks.ModelCheckpoint("LR_best_model.weights.hdf5", 
                    monitor="val_accuracy", mode="max", 
                    save_best_only=True, verbose=1)

    history = resnet_model.fit(train_ds, epochs=40, callbacks=[callback, checkpoint], validation_data = test_ds, verbose=0)
    
    resnet_model.load_weights("LR_best_model.weights.hdf5")

    ##################################################
    import numpy as np

    y_pred = []
    y_true = []

    for img, labels in test_ds:
        group_labels = resnet_model.predict(img,verbose=0)
        for label in group_labels.tolist():
            y_pred.append(np.round(label[0]))
        labels = labels.numpy()
        for label in labels.tolist():
            y_true.append(label[0])

    ###################################################
    import csv
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    # Save values to CSV file
    header = ['Accuracy', 'Precision', 'Recall', 'F1-score']
    data = [accuracy, precision, recall, f1]

    csv_filename = 'performance_metrics\LR\cross_validation_{}_results.csv'.format(i)

    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerow(data)

    print(f'Metrics saved to {csv_filename}')

    ##################################################
    #Saving epoch and accuracy results into  a csv file
    counter = 1
    Accuracy = []
    Epoch = []
    Val_accuracy = []

    for value in history.history['accuracy']:
        Accuracy.append(round(value,2))
        Epoch.append(counter)
        counter += 1

    for value in history.history['val_accuracy']:
        Val_accuracy.append((round(value,2)))

    import csv

    csv_file = 'Epoch_SR_results\LR\cross_validation_{}_epcoch_SR_results.csv'.format(i)

    # Combine the two lists into a list of tuples
    data = zip(Epoch, Accuracy, Val_accuracy)

    # Write to the CSV file
    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)

        # Write the header
        writer.writerow(['Epoch', 'Accuracy', 'Val_accuracy'])

        # Write the data
        writer.writerows(data)

    print(f'CSV file "{csv_file}" created successfully.')


########################################################

import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt 

#Calculating the confusion matrix.
matrix = confusion_matrix(y_pred,y_true)
 
#Computing the confusion matrix.
sns.heatmap(matrix, 
            annot=True,
            fmt='g', 
            xticklabels=['HGG', 'LGG'],
            yticklabels=['HGG','LGG'])
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.title('Confusion Matrix',fontsize=17)
plt.savefig('LR_plot.png')
plt.show()